In [6]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import transforms
import glob
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
import random
import cv2
import sys

In [ ]:
def load_from_dataset (path , img_size_w , img_size_h):
  data = []
  for f in glob.iglob(path):
    img = cv2.imread(f , cv2.IMREAD_COLOR)
    img = cv2.resize(img,(img_size_w,img_size_h))
    data.append(img)
  return data

In [8]:
class Data(Dataset):
    def __init__(self, img_size_w = 224, img_size_h = 224, mode='train', train_transform=None, val_transform=None):
        self.img_size_w = img_size_w

        self.img_size_h = img_size_h

        self.X_train, self.y_train, self.X_test, self.y_test = None, None, None, None

        self.mode = mode

        benign_path = "../data/benign/*.jpg"
        malignant_path = "../data/malignant/*.jpg"

        benign = load_from_dataset (benign_path , img_size_w , img_size_h)
        malignant = load_from_dataset (malignant_path , img_size_w , img_size_h)

        benign = np.array(benign , dtype=np.float32)
        malignant = np.array(malignant , dtype=np.float32)

        benign_labels = np.zeros(benign.shape[0], dtype=np.float32)
        malignant_labels = np.full(malignant.shape[0], 1.0)

        self.images = np.concatenate((benign, malignant), axis=0)
        self.labels = np.concatenate((benign_labels, malignant_labels))

        self.train_transform = train_transform if train_transform else transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((img_size_h, img_size_w)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
            #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        self.val_transform = val_transform if val_transform else transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((img_size_h, img_size_w)),
            transforms.ToTensor(),
            #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def train_val_split(self):
        self.X_train, self.X_test, self.y_train, self.y_test = \
        train_test_split(self.images, self.labels, test_size=0.20, random_state=42)

    def __len__(self):
        if self.mode == 'train':
          return self.X_train.shape[0]
        elif self.mode == 'val':
          return self.X_test.shape[0]

    def __getitem__(self, idx):
        if self.mode == 'train':
            image, label = self.X_train[idx], self.y_train[idx]
            image = self.train_transform(image)
        elif self.mode == 'val':
            image, label = self.X_test[idx], self.y_test[idx]
            image = self.val_transform(image)

        label = torch.tensor(label, dtype=torch.float32)
        return {'image': image, 'label': label}

    def normalize(self):
        self.images = self.images/255.0